In [69]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import itertools
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression


from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

kB = 1000 * Boltzmann / electron_volt

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

In [74]:
cif_filepath1 = "data/mn8au16_collinear.cif"
cif_filepath2 = "data/mn8au16_1x1x4.cif"
structure_mnau2_c = from_file(structure_file=cif_filepath1)
structure_mnau2_z = from_file(structure_file=cif_filepath2)

def pattern_tables( layer, p_type):
    n_row, n_column = layer.shape
    au_layer = pd.DataFrame(np.zeros((2,n_column)))
    patterns = pd.concat([ layer[0:1],au_layer,
                           layer[1:2],au_layer,
                           layer[2:3],au_layer,
                           layer[3:4],au_layer,
                           layer[4:5],au_layer,
                           layer[5:6],au_layer,
                           layer[6:7],au_layer,
                           layer[7:8],au_layer], ignore_index=True)
    
    p_types =[]
    p_types.append("nm")
    index = np.arange(1,len(patterns.columns))
    for i in index:
        p_types.append(p_type+str(i))
    patterns.columns = p_types
    
    return patterns

def pattern_model(pattern_structure, pattern_mg):
    pattern_neighbor_data = count_neighbors(cell_structure=pattern_structure, r=8.8)
    exchange_model = build_model(magnetic_patterns=pattern_mg, neighbor_data=pattern_neighbor_data)
    #drop the Au atoms
    exchange_model = exchange_model[exchange_model.columns.drop(list(exchange_model.filter(regex='Au')))]
    
    return exchange_model


/home/ssilayi/anaconda3/lib/python3.6/site-packages/pymatgen/core/lattice.py:694: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(q.T, p.T)[0].T


In [120]:
c_layer=pd.read_csv('data/mg_collinear_patterns', header=None)
c_patterns = pattern_tables( c_layer, "c-")

neighbor_data = count_neighbors(cell_structure=structure_mnau2_c, r=17.6)
a = neighbor_data.data_frame \
    .merge(neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
     .query("subspecies_i != 'Au'")\
        .query("subspecies_j != 'Au'")\
    .reset_index(drop=True)

type(a.distance_bin)

pandas.core.series.Series